In [ ]:
import os
import asyncio
import nest_asyncio

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [ ]:
from se489_group_project.constants import *
from se489_group_project.utility.common import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_storage])


    
    def get_training_config(self) -> ModelTrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = ModelTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import webbrowser
import subprocess
import torch
from torch.profiler import profile, record_function, ProfilerActivity


In [ ]:
import mlflow 
import mlflow.keras
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import  accuracy_score, roc_auc_score, log_loss, confusion_matrix, balanced_accuracy_score, matthews_corrcoef


In [ ]:
class MetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_data = self.validation_data
        val_pred = []
        val_true = []

        for batch in range(len(val_data)):
            x_val, y_val = val_data[batch]
            y_pred = self.model.predict(x_val)
            val_pred.extend(tf.argmax(y_pred, axis=1).numpy())
            val_true.extend(tf.argmax(y_val, axis=1).numpy())

        precision = precision_score(val_true, val_pred, average='weighted')
        recall = recall_score(val_true, val_pred, average='weighted')
        f1 = f1_score(val_true, val_pred, average='weighted')

        logs['val_precision'] = precision
        logs['val_recall'] = recall
        logs['val_f1'] = f1
        print(f' - val_precision: {precision:.4f} - val_recall: {recall:.4f} - val_f1: {f1:.4f}')


In [ ]:
class Training:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    
    async def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    async def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    async def stop_profiler_if_running(self):
        """Stops the profiler if it is currently running."""
        try:
            tf.profiler.experimental.stop()
        except tf.errors.UnimplementedError:
            print("Profiler was not running.")
        except tf.errors.FailedPreconditionError:
            print("No profiler is running.")
        except Exception as e:
            print(f"An unexpected error occurred while stopping the profiler: {e}")

    
    async def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        log_dir = "se489_group_project/visualizations/tensorboard_logs"
        
        os.makedirs(log_dir, exist_ok=True)
       
        # Stop any running profiler instances before starting a new one
        await self.stop_profiler_if_running()
        mlflow.set_experiment("se489_group_project")
        with mlflow.start_run():
            mlflow.keras.autolog()
        #TensorFlow Profiling 
        tf.profiler.experimental.start(log_dir)

         # Set up TensorBoard callback
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            profile_batch='500,520'  # Profile batches 500 to 520
        )
        #Make sure this is necessary now that code is working
        #   # Recreate the optimizer instance each time before compiling
        # optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

        try:
            # self.model.compile(
            #     optimizer=optimizer,
            #     loss=tf.keras.losses.CategoricalCrossentropy(reduction='sum_over_batch_size'),
            #     metrics=["accuracy"]
            # )
            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator,
                callbacks=[tensorboard_callback]
            )
             # Predict on validation data
            y_true = self.valid_generator.classes
            y_pred = self.model.predict(self.valid_generator)
            y_pred_classes = tf.argmax(y_pred, axis=1).numpy()

                # Calculate and log metrics
            metrics = {
                    'f1_score': f1_score(y_true, y_pred_classes, average='weighted'),
                    'precision_score': precision_score(y_true, y_pred_classes, average='weighted'),
                    'recall_score': recall_score(y_true, y_pred_classes, average='weighted'),
                    'accuracy': accuracy_score(y_true, y_pred_classes),
                    'roc_auc_score': roc_auc_score(y_true, y_pred_classes, multi_class='ovr'),
                    'log_loss': log_loss(y_true, y_pred_classes),
                    'balanced_accuracy': balanced_accuracy_score(y_true, y_pred_classes),
                    'matthews_corrcoef': matthews_corrcoef(y_true, y_pred_classes)
                }

            for metric_name, metric_value in metrics.items():
                mlflow.log_metric(metric_name, metric_value)
            
        finally:
        
            tf.profiler.experimental.stop()
        
       
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
    

In [ ]:
import cProfile
import webbrowser
#async added to main function to improve performance metrics
async def main():
        logs = "se489_group_project/visualizations/tensorboard_logs"
        
        subprocess.Popen(["tensorboard", "--logdir", logs, "--host", "0.0.0.0", "--port", "6006"])
        time.sleep(5)
        webbrowser.open("http://localhost:6006/")

        log_dir = os.path.join(os.getcwd(), "se489_group_project", "visualizations")
        file = os.path.join(log_dir, 'cprofile_stats_model_training.prof')
        config = ConfigurationManager()
        training_config = config.get_training_config()
        training = Training(config=training_config)
        profiler = cProfile.Profile()
        profiler.enable()
        await training.get_base_model()
        await training.train_valid_generator()
        await training.train()
        profiler.disable()
        profiler.dump_stats(file)
        profile_file_full_path = os.path.abspath(file)
        
        #Automatically open snakeviz to visualize the profiling results
        try:
            subprocess.Popen(["snakeviz", profile_file_full_path])
        except FileNotFoundError:
            print("snakeviz is not installed or not found in the system path.")
    

if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())

